In [1]:
%reset -f

In [2]:
%reload_ext sql
%sql sqlite:///../ontology/tmp/merged.db

In [3]:
%sql SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///../ontology/tmp/merged.db
Done.


name
term_association
has_oio_synonym_statement
anonymous_expression
anonymous_class_expression
anonymous_property_expression
anonymous_individual_expression
owl_restriction
owl_complex_axiom
prefix
statements


In [28]:
import pandas as pd

def save_csv(query, fn):
    df = pd.DataFrame(query)
    df.to_csv(fn, sep='\t', index = False)
    return df.head()

In [29]:
query = %sql SELECT * FROM statements WHERE subject LIKE "%MONDO:%" AND object LIKE "%ICD10%"
save_csv(query, "mondo_2_icd.tsv")

 * sqlite:///../ontology/tmp/merged.db
Done.


,stanza,subject,predicate,object,value,datatype,language
0,MONDO:0000087,MONDO:0000087,skos:narrowMatch,<https://icd.codes/icd10cm/Q04.3>,None,None,None
1,MONDO:0000088,MONDO:0000088,skos:exactMatch,<https://icd.codes/icd10cm/E30.1>,None,None,None
2,MONDO:0000088,MONDO:0000088,skos:exactMatch,<http://purl.bioontology.org/ontology/ICD10CM/...,None,None,None
3,MONDO:0000107,MONDO:0000107,skos:narrowMatch,<https://icd.codes/icd10cm/Q75.8>,None,None,None
4,MONDO:0000110,MONDO:0000110,skos:narrowMatch,<https://icd.codes/icd10cm/Q30.2>,None,None,None


In [30]:
query = %sql SELECT * FROM statements WHERE subject LIKE "%MONDO:%" OR subject LIKE "%ICD10%" AND object LIKE "%MONDO:%" OR object LIKE "%ICD10%" LIMIT 10
save_csv(query, "mondo_icd_2_mondo_icd.tsv")  

 * sqlite:///../ontology/tmp/merged.db
Done.


,stanza,subject,predicate,object,value,datatype,language
0,MONDO:0100332,MONDO:0100332,rdfs:label,None,disease has primary infectious agent,xsd:string,None
1,MONDO:0100332,MONDO:0100332,oio:id,None,MONDO:0100332,xsd:string,None
2,MONDO:0100332,MONDO:0100332,oio:created_by,None,https://orcid.org/0000-0001-5208-3432,xsd:string,None
3,MONDO:0100332,MONDO:0100332,rdfs:subPropertyOf,RO:0014001,None,None,None
4,MONDO:0100332,MONDO:0100332,rdf:type,owl:ObjectProperty,None,None,None


In [36]:
query = %sql SELECT * FROM statements WHERE (subject LIKE "%MONDO:%" OR subject LIKE "%icd10cm%") AND predicate IN ("rdfs:label", "oio:hasExactSynonym")
save_csv(query, "mondo_icd_label_exact_synonym.tsv")


 * sqlite:///../ontology/tmp/merged.db
Done.


,stanza,subject,predicate,object,value,datatype,language
0,<http://purl.bioontology.org/ontology/ICD10CM/...,<http://purl.bioontology.org/ontology/ICD10CM/...,oio:hasExactSynonym,None,diseases generally recognized as communicable ...,None,en
1,<http://purl.bioontology.org/ontology/ICD10CM/...,<http://purl.bioontology.org/ontology/ICD10CM/...,oio:hasExactSynonym,None,Classical cholera,None,en
2,<http://purl.bioontology.org/ontology/ICD10CM/...,<http://purl.bioontology.org/ontology/ICD10CM/...,oio:hasExactSynonym,None,Cholera eltor,None,en
3,<http://purl.bioontology.org/ontology/ICD10CM/...,<http://purl.bioontology.org/ontology/ICD10CM/...,oio:hasExactSynonym,None,Infection due to Salmonella typhi,None,en
4,<http://purl.bioontology.org/ontology/ICD10CM/...,<http://purl.bioontology.org/ontology/ICD10CM/...,oio:hasExactSynonym,None,Typhoid myocarditis,None,en


In [6]:
%sql SELECT * FROM statements WHERE (subject LIKE "%icd10cm%") AND predicate IN ("rdfs:label", "oio:hasExactSynonym") LIMIT 100


 * sqlite:///../ontology/tmp/merged.db
Done.


stanza,subject,predicate,object,value,datatype,language
<http://purl.bioontology.org/ontology/ICD10CM/A00-B99>,<http://purl.bioontology.org/ontology/ICD10CM/A00-B99>,oio:hasExactSynonym,None,diseases generally recognized as communicable or transmissible,None,en
<http://purl.bioontology.org/ontology/ICD10CM/A00.0>,<http://purl.bioontology.org/ontology/ICD10CM/A00.0>,oio:hasExactSynonym,None,Classical cholera,None,en
<http://purl.bioontology.org/ontology/ICD10CM/A00.1>,<http://purl.bioontology.org/ontology/ICD10CM/A00.1>,oio:hasExactSynonym,None,Cholera eltor,None,en
<http://purl.bioontology.org/ontology/ICD10CM/A01.0>,<http://purl.bioontology.org/ontology/ICD10CM/A01.0>,oio:hasExactSynonym,None,Infection due to Salmonella typhi,None,en
<http://purl.bioontology.org/ontology/ICD10CM/A01.02>,<http://purl.bioontology.org/ontology/ICD10CM/A01.02>,oio:hasExactSynonym,None,Typhoid myocarditis,None,en
<http://purl.bioontology.org/ontology/ICD10CM/A01.02>,<http://purl.bioontology.org/ontology/ICD10CM/A01.02>,oio:hasExactSynonym,None,Typhoid endocarditis,None,en
<http://purl.bioontology.org/ontology/ICD10CM/A01.4>,<http://purl.bioontology.org/ontology/ICD10CM/A01.4>,oio:hasExactSynonym,None,Infection due to Salmonella paratyphi NOS,None,en
<http://purl.bioontology.org/ontology/ICD10CM/A02>,<http://purl.bioontology.org/ontology/ICD10CM/A02>,oio:hasExactSynonym,None,infection or foodborne intoxication due to any Salmonella species other than S. typhi and S. paratyphi,None,en
<http://purl.bioontology.org/ontology/ICD10CM/A02.0>,<http://purl.bioontology.org/ontology/ICD10CM/A02.0>,oio:hasExactSynonym,None,Salmonellosis,None,en
<http://purl.bioontology.org/ontology/ICD10CM/A02.25>,<http://purl.bioontology.org/ontology/ICD10CM/A02.25>,oio:hasExactSynonym,None,Salmonella tubulo-interstitial nephropathy,None,en


In [7]:
%sql SELECT * FROM entailed_edge WHERE object LIKE "%ICD10%" LIMIT 100

 * sqlite:///../ontology/tmp/merged.db
Done.


subject,predicate,object
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M60-M63>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M61.05>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M61.0>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M00-M99>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M61.052>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M61>
<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1A>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/T36.3>
<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1A>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1>
<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1A>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/T36-T50>
<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1A>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/S00-T88>


In [34]:
%sql SELECT * FROM entailed_edge WHERE subject LIKE "%MONDO_%" AND object LIKE "%ICD10%" LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


subject,predicate,object


In [33]:
%sql SELECT * FROM entailed_edge WHERE subject LIKE "%ICD10%" AND object LIKE "%MONDO_%"  LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


subject,predicate,object


In [32]:
%sql SELECT * FROM entailed_edge WHERE subject LIKE "%ICD10%" LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


subject,predicate,object
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M60-M63>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M61.05>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M61.0>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M00-M99>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M61.052>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M61>
<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1A>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/T36.3>
<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1A>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1>
<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1A>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/T36-T50>
<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1A>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/S00-T88>
